In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

import importlib
import os

from matplotlib import pyplot as plt
from sqlalchemy import create_engine

from ppcollapse import setup_logger
from ppcollapse.cvat_to_db import create_collapses_from_cvat
from ppcollapse.setup_django_ppcx import setup_django
from ppcollapse.utils.config import ConfigManager
from ppcollapse.utils.database import get_collapses_df, get_image

# Allow Django ORM to work in Jupyter's async environment
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Setup Django
config = ConfigManager(config_path="config.yaml")
setup_django(db_config=config.get("database"))

logger = setup_logger(level="INFO", name="ppcx")
config = ConfigManager(config_path="config.yaml")
db_engine = create_engine(config.db_url)

# Import Django models
ppcx_app_models = importlib.import_module("ppcx_app.models")
Collapse = ppcx_app_models.Collapse
Image = ppcx_app_models.Image

Django is already configured. Skipping setup_django().


Manual fix of some collapse dates


In [4]:
# 09/10/2017 - volume wrong (should be 3500 m3)

skip = True
if not skip:
    collapse_id = 1122
    collapse = Collapse.objects.get(id=collapse_id)
    collapse.volume = 3500
    collapse.save()

In [6]:
# Create new collapse from cvat data labels
skip = True
if not skip:
    created_ids = create_collapses_from_cvat(
        "data/job_23_annotations_2025_11_25_17_21_42_cvat for images 1.1.xml",
        label_name="collapse",
        image_search_mode="auto",
        time_tolerance_seconds=300,
        remove_reg_suffix=True,
        dry_run=True,
    )

## OLD CELL


In [ ]:
# List of collapse ID to delete
collapse_ids_to_delete = [
    49,
    50,
    56,
    57,
    61,
    62,
    67,
    71,
    77,
    79,
    81,
    83,
    84,
    91,
    93,
    94,
    101,
    104,
    113,
    117,
    118,
    119,
    120,
    121,
    122,
    123,
    125,
    126,
    129,
    130,
    133,
    137,
    139,
    141,
    145,
    147,
    148,
    149,
    150,
    160,
    168,
    169,
    171,
    173,
    174,
    177,
    178,
    182,
    188,
]

In [ ]:
engine = create_engine(config.db_url)
df = get_collapses_df(engine)
if df.empty:
    logger.warning("No collapses found in database.")
df

In [ ]:
from shapely.ops import unary_union

# Example: merge multiple polygons from your collapse dataframe

collapse_1 = df.iloc[0]
collapse_2 = df.iloc[1]

geom1 = shapely_wkt.loads(collapse_1["geom_wkt"])
geom2 = shapely_wkt.loads(collapse_2["geom_wkt"])

# Merge into single geometry (will be MultiPolygon if not contiguous)
merged_geom = unary_union([geom1, geom2])

# If they ARE contiguous, this will return a Polygon
# If they're NOT contiguous, this will return a MultiPolygon
print(type(merged_geom))  # Polygon or MultiPolygon


In [ ]:
image = get_image(image_id=int(collapse_1["image_id"]), config=config)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image)
# Handle both Polygon and MultiPolygon
if merged_geom.geom_type == "Polygon":
    # Single polygon - plot directly
    xs, ys = merged_geom.exterior.xy
    ax.plot(xs, ys, color="red", linewidth=2)
    ax.fill(xs, ys, facecolor="red", edgecolor="none", alpha=0.3)
elif merged_geom.geom_type == "MultiPolygon":
    # Multiple polygons - iterate and plot each
    for poly in merged_geom.geoms:
        xs, ys = poly.exterior.xy
        ax.plot(xs, ys, color="red", linewidth=2)
        ax.fill(xs, ys, facecolor="red", edgecolor="none", alpha=0.3)
else:
    raise ValueError("Merged geometry is neither Polygon nor MultiPolygon.")
ax.set_axis_off()
ax.set_title("Collapse Geometry", fontsize=10, pad=5)

In [ ]:
# Now import Django models
from ppcx_app.models import Collapse
from shapely import wkt as shapely_wkt


In [ ]:
# Get the merged geometry
merged_geom = unary_union([geom1, geom2])

# Convert Shapely geometry to Django GEOSGeometry
wkt_str = shapely_wkt.dumps(merged_geom)
django_geom = GEOSGeometry(wkt_str, srid=0)

collapse_obj = Collapse.objects.get(id=collapse_1["id"])
collapse_obj.geom = django_geom
